In [18]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright
import requests
from bs4 import BeautifulSoup
import pandas as pd
nest_asyncio.apply()

In [20]:
# Cargamos la información necesaria para el procesamiento
base_creditos = pd.read_excel('../01_Datos/solicitudes.xlsx')
Valor_simular = base_creditos.copy()

In [22]:
Valor_simular = Valor_simular.rename(columns=lambda x: x.replace(' ', ''))
Valor_simular

,ValorCredito,Plazo
0,2000000,12
1,2000000,24
2,2000000,36
3,2000000,48
4,3000000,24
5,3000000,36
6,4500000,6
7,4500000,24
8,10000000,24
9,10000000,36


In [28]:
import pandas as pd
import asyncio
from playwright.async_api import async_playwright

async def simular_creditos(df):
    # Lista para almacenar los resultados
    resultados = []
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # Visible para depuración
        page = await browser.new_page()
        await page.goto('https://www.bancodeoccidente.com.co/solicitarcredito/#/simuladorCreditoConsumo')
        
        for index, row in df.iterrows():
            valor_credito = str(row['ValorCredito'])
            plazo = str(row['Plazo'])
            print(f"Fila {index + 1}/{len(df)}: Simulando ValorCredito={valor_credito}, Plazo={plazo}")
            
            try:
                await asyncio.sleep(2)
                await page.wait_for_selector("a[ng-click=\"laf.irEstado('irSimuladorCreditoConsumo')\"]", timeout=10000)
                await asyncio.sleep(2)
                await page.click("a[ng-click=\"laf.irEstado('irSimuladorCreditoConsumo')\"]")
                await asyncio.sleep(2)
                await page.wait_for_selector("input[name='monto']", timeout=10000)
                await page.click("input[name='monto']", force=True)
                await page.fill("input[name='monto']", "")
                await page.fill("input[name='monto']", valor_credito)
                await page.wait_for_selector("select[name='plazo']", timeout=10000)
                await page.click("select[name='plazo']")
                await page.select_option("select[name='plazo']", value=plazo)
                await page.wait_for_selector("span#resultado b.ng-binding", timeout=15000)
                valor_cuota = await page.eval_on_selector("span#resultado b.ng-binding", "el => el.innerText")
                
                resultados.append({
                    'ValorCredito': valor_credito,
                    'Plazo': plazo,
                    'valor_cuota': valor_cuota
                })
                print(f"Resultado: {valor_cuota}")
                
            except Exception as e:
                print(f"Fila {index + 1}: No se pudo realizar la simulación ({str(e)})")
                resultados.append({
                    'ValorCredito': valor_credito,
                    'Plazo': plazo,
                    'valor_cuota': 'No se pudo realizar la simulación'
                })
            
            await page.goto('https://www.bancodeoccidente.com.co/solicitarcredito/#/')
            await asyncio.sleep(2)
        
        await browser.close()
    
    # Crear el DataFrame de resultados
    df_resultados = pd.DataFrame(resultados)
    return df_resultados

# Ejecutar la simulación
async def main():
    if 'Valor_simular' not in globals():
        raise ValueError("El DataFrame 'Valor_simular' no está definido. Cárgalo primero.")
    
    print("DataFrame original:")
    print(Valor_simular)
    
    # Ejecutar la simulación y asignar a una variable global
    global df_resultados_global
    df_resultados_global = await simular_creditos(Valor_simular)
    
    # Mostrar resultados
    print("\nResultados de la simulación:")
    print(df_resultados_global)
    
    # Guardar como CSV y Pickle para acceso desde otros notebooks
    df_resultados_global.to_csv('resultados_simulacion.csv', index=False)
    #df_resultados_global.to_pickle('resultados_simulacion.pkl')
    #print("Resultados guardados en 'resultados_simulacion.csv' y 'resultados_simulacion.pkl'")

# Correr el script
if __name__ == "__main__":
    asyncio.run(main())

DataFrame original:
   ValorCredito  Plazo
0       2000000     12
1       2000000     24
2       2000000     36
3       2000000     48
4       3000000     24
5       3000000     36
6       4500000      6
7       4500000     24
8      10000000     24
9      10000000     36
Fila 1/10: Simulando ValorCredito=2000000, Plazo=12
Resultado: $196.038
Fila 2/10: Simulando ValorCredito=2000000, Plazo=24
Resultado: $112.941
Fila 3/10: Simulando ValorCredito=2000000, Plazo=36
Resultado: $86.091
Fila 4/10: Simulando ValorCredito=2000000, Plazo=48
Fila 4: No se pudo realizar la simulación (Page.select_option: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("select[name='plazo']")
  -     - locator resolved to 2 elements. Proceeding with the first one: <select id="plazo" required="" name="plazo" ng-model="scc.simulacion.plazo" class="form-control input-sm ng-pristine ng-untouched ng-valid ng-not-empty ng-valid-required">…</select>
  -   - attempting select option action
  -     2 × waitin

In [26]:
df_resultados_global

,ValorCredito,Plazo,valor_cuota
0,2000000,12,$196.038
1,2000000,24,$112.941
2,2000000,36,$86.091
3,2000000,48,No se pudo realizar la simulación
4,3000000,24,$169.411
5,3000000,36,$129.136
6,4500000,6,No se pudo realizar la simulación
7,4500000,24,$254.116
8,10000000,24,$564.703
9,10000000,36,$430.453


Oportunidades de mejora
- Un try catch para validar la disponibilidad de la pagina para descarga
- Validar y almacenar los notebook para guardar como funciones y guardarlo con extensión py para extraer la data
- Realizar el análisis descriptivo de la data
- Validar cual va a ser el procedimiento y definir el paso a paso

Dudas